<a href="https://colab.research.google.com/github/Homaoa/NLP-Using-spaCy-for-a-Hotel-Reviews-Data-Set/blob/main/NLP_using_spaCy_and_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

2023-11-14 20:39:39.527360: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 20:39:39.527419: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 20:39:39.527449: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 20:39:39.533682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 20:39:40.513148: W tensorflow/compiler/

In [2]:
import pandas as pd

In [3]:
# I use the hotel reviews data set that I used in my other NLP project (NLP-for-a-Hotel-Reviews-Data-Set), using NLTK.
# In this code, I take predictions for the model

url = "https://raw.githubusercontent.com/Homaoa/NLP-for-a-Hotel-Reviews-Data-Set/main/tripadvisor_hotel_reviews.csv"
df = pd.read_csv(url)
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [6]:
df['Rating'].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [ ]:
# Ratings are between 1 to 5

In [5]:
# Now I can import spaCy and define nlp

import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
# I define X and y

X = df["Review"]
y = df["Rating"]

In [8]:
# Now I split the data into train and test to apply classification models

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [10]:
# I build the word vector for Reviews

docs = [nlp(text) for text in X_train]
X_train_word_vectors = [x.vector for x in docs]

In [24]:
# The firsl algorithm I use is SVM

from sklearn import svm
classifier = svm.SVC(kernel='linear')
classifier.fit(X_train_word_vectors, y_train)

SVC(kernel='linear')

In [25]:
test_docs = [nlp(text) for text in X_test]
X_test_word_vectors =  [x.vector for x in test_docs]
y_pred = classifier.predict(X_test_word_vectors)

In [26]:
from sklearn.metrics import confusion_matrix , accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[ 111   45    0   18  111]
 [  40   83    0   57  175]
 [  24   49    0  140  258]
 [  12   29    0  270  892]
 [  27   17    0  152 1589]]


0.5008538667967797

In [27]:
# I try logistics regression algorithm now to classify

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_word_vectors, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [28]:
y_pred = classifier.predict(X_test_word_vectors)
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[ 113   42   10   21   99]
 [  45   71   21   87  131]
 [  23   38   33  165  212]
 [  11   25   31  385  751]
 [  22   19    8  253 1483]]


0.5086606489387655

In [29]:
# Now I train the naive bayes model on the training set and get prediction

from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(X_train_word_vectors,y_train)

GaussianNB()

In [30]:
y_pred = classifier.predict(X_test_word_vectors)
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[145  74  20  11  35]
 [ 88 156  32  47  32]
 [ 56 134  62 118 101]
 [ 55 247 106 395 400]
 [105 261  97 470 852]]


0.3927787265186631

In [22]:
# Now I try knn model

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train_word_vectors, y_train)

KNeighborsClassifier()

In [31]:
y_pred = classifier.predict(X_test_word_vectors)
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[145  74  20  11  35]
 [ 88 156  32  47  32]
 [ 56 134  62 118 101]
 [ 55 247 106 395 400]
 [105 261  97 470 852]]


0.3927787265186631